

# 데이터 크롤링
# 


In [2]:
## 모듈 가져오기

import pandas as pd
import requests
import time
import html5lib


In [3]:
import os

os.getcwd()

'C:\\Users\\82108\\Downloads\\KQR\\페이즈4'

# 전 종목 코드 구해오기


In [3]:
# pykrx를 이용하여 전종목코드와 기업명을 가져옴

from pykrx import stock
tickers = stock.get_market_ticker_list(market='ALL') #pykrx의 stock클래스의 종목코드 불러오기(전체)
Name=[]
for ticker in tickers:
        Name.append(stock.get_market_ticker_name(ticker)) #반복문을 이용해 모든 종목 코드들을 리스트안에 전부 할당
        
#데이터 프레임 형태로 만들기
code_data=pd.DataFrame()
code_data['종목코드']=tickers 
code_data['기업명'] = Name
code_data['종목코드']='A'+code_data['종목코드']
code_data

,종목코드,기업명
0,A060310,3S
1,A095570,AJ네트웍스
2,A006840,AK홀딩스
3,A054620,APS홀딩스
4,A265520,AP시스템
...,...,...
2567,A000547,흥국화재2우B
2568,A000545,흥국화재우
2569,A003280,흥아해운
2570,A037440,희림


###  데이터프레임 형태 바꾸기 함수 -- 크롤링한 데이터프레임의 형식을 바꿔줌

In [4]:

def change_df(firm_code, dataframe): #종목코드와 해당 데이터 프레임을 매개변수로 가짐
    
    for num, col in enumerate(dataframe.columns):
        temp_df = pd.DataFrame({firm_code : dataframe[col]})
        temp_df = temp_df.T
        temp_df.columns = [[col]*len(dataframe), temp_df.columns]
        
        if num == 0:
            total_df = temp_df
        else:
            total_df = pd.merge(total_df, temp_df, how='outer', left_index=True, right_index=True)    
    
    return total_df

# 시가총액 데이터 가져오기

중형주 : 시가총액 101위에서 300위 사이를 뜻한다

In [5]:
#시가총액 데이터를 가져오는 함수

def make_allstock_dataframe(firm_code): #종목명을 매개변수로 사용
    allstock_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Main.asp?pGB=1&gicode=' + firm_code #url을 할당
    allstock_page = requests.get(allstock_url) #request의 해당 url의 페이지 데이터를 호출
    allstock_tables = pd.read_html(allstock_page.text) #html파일을 텍스트 파일 형태로 가져와 데이터프레임으로 만듦
    temp_df = allstock_tables[0] #해당 내용이 들어있는 테이블 위치
    
    temp_df = temp_df.set_index(temp_df.columns[0]) #데이터테이블의 0번째 열을 인덱스로 사용
    temp_df = temp_df.loc[['시가총액(보통주,억원)']] # '시가총액' 인덱스의 데이터를 "데이터프레임 형식으로 가져옴" using [[]] (겹괄호)
    temp_df.index = ['시가총액'] #'시가총액'이라는 이름으로 인덱스 이름 변경
    temp_df = temp_df[[1]] # 다시말하지만 겹괄호를 사용하면 데이터프레임 형식으로 출력
    return temp_df

In [ ]:
#시가총액 가져오기 -- PYKRX로 가져온 종목코드를 이용함

for num, code in enumerate(code_data['종목코드']): #코스피 시가총액구하기 # enumerate로 종목코드의 순서와 이름을 둘다 가져올수 있음
    try:                                                                  # ex)  (1, A000000)
        print(num, code)   #해당 순번과 종목코드 출력
        time.sleep(0.25)      #2초간 텀을 둠
        
        try:
            allstock_df = make_allstock_dataframe(code) #시가총액데이터를 가져오기
        except requests.exceptions.Timeout: #예외상황 -- 시간초과
            time.sleep(60)
            allstock_df = make_allstock_dataframe(code) #다시 시도
            
        allstock_df_changed = change_df(code, allstock_df) #데이터 프레임 형태 바꾸기(함수)
        
        if num == 0 :
            total_allstock = allstock_df_changed
        else:
            total_allstock = pd.concat([total_allstock, allstock_df_changed])
    except ValueError:
        continue
    except KeyError:
        continue

0 A060310
1 A095570
2 A006840
3 A054620
4 A265520
5 A211270
6 A027410
7 A282330
8 A138930
9 A001460
10 A001465
11 A013720
12 A001040
13 A079160
14 A035760
15 A00104K
16 A000120
17 A011150
18 A011155
19 A001045
20 A097950
21 A097955
22 A051500
23 A058820
24 A023460
25 A056730
26 A056000
27 A065770
28 A083660
29 A000590
30 A012030
31 A367340
32 A367360
33 A016610
34 A005830
35 A000990
36 A000995
37 A139130
38 A001530
39 A000210
40 A001880
41 A000215
42 A375500
43 A37550K
44 A068790
45 A004840
46 A241520
47 A155660
48 A069730
49 A017940
50 A245620
51 A037370
52 A278990
53 A079190
54 A365550
55 A050120
56 A383220
57 A007700
58 A130500
59 A114090
60 A900290
61 A078930
62 A083450
63 A006360
64 A001250
65 A007070
66 A078935
67 A045890
68 A297890
69 A078150
70 A012630
71 A039570
72 A089470
73 A294870
74 A011200
75 A036640
76 A082740
77 A335870
78 A351340
79 A351320
80 A373200
81 A003560
82 A095340
83 A099520
84 A175330
85 A950170
86 A234080
87 A067290
88 A001060
89 A001067


In [ ]:
total_allstock

In [8]:
#시가총액 데이터저장
total_allstock.to_csv('시가총액.csv')

In [9]:
#중형주 리스트 가져오기

mediumlist = pd.read_csv("시가총액.csv")

mediumlist1 = pd.DataFrame()

mediumlist1['종목코드'] = mediumlist.iloc[1:,0]
mediumlist1['시가총액'] = mediumlist.iloc[1:,1].apply(pd.to_numeric)

mediumlist = mediumlist1

mediumlist = mediumlist.sort_values(by = ['시가총액'],ascending=False) #내림차순 정렬

mediumlist = mediumlist[100:300] #100위 부터 299위까지

mediumlist = mediumlist.reset_index(drop = True)

In [10]:
mediumlist

,종목코드,시가총액
0,A196170,35093.0
1,A010620,35029.0
2,A036460,34571.0
3,A036490,34280.0
4,A039490,34048.0
...,...,...
195,A010050,9135.0
196,A268280,9104.0
197,A280360,9080.0
198,A082640,9052.0


In [12]:
mediumlist.to_csv('중형주리스트.csv')

# 중형주 리스트 가져오기

In [4]:
mediumlist = pd.read_csv("중형주리스트.csv")
mediumlist

,Unnamed: 0,종목코드,시가총액
0,0,A196170,35093.0
1,1,A010620,35029.0
2,2,A036460,34571.0
3,3,A036490,34280.0
4,4,A039490,34048.0
...,...,...,...
395,395,A018000,4869.0
396,396,A001510,4868.0
397,397,A321550,4860.0
398,398,A002310,4845.0


# PER 데이터 가져오기 - kqr 데이터수집 코드 참고

In [ ]:
# 투자지표 데이터프레임을 만드는 함수 (데이터 수집하기_재무 데이터 구해오기)

def make_invest_dataframe(firm_code):
    invest_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode=' + firm_code
    invest_page = requests.get(invest_url)
    invest_tables = pd.read_html(invest_page.text)
    temp_df = invest_tables[1]
    
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['PER계산에 참여한 계정 펼치기']]
    temp_df.index = ['PER']
    return temp_df

In [ ]:
for num, code in enumerate(mediumlist['종목코드']): #중형주 per 가져오기
    try:
        print(num, code)
        time.sleep(1)
        try:
            invest_df = make_invest_dataframe(code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            invest_df = make_invest_dataframe(code)
        invest_df_changed = change_df(code, invest_df)
        if num == 0 :
            total_invest = invest_df_changed
        else:
            total_invest = pd.concat([total_invest, invest_df_changed])
    except ValueError:
        continue
    except KeyError:
        continue

In [ ]:
total_invest

In [ ]:
total_invest.to_csv('PER.csv')#per 데이터 저장

# EPS 증가율 데이터 가져오기

In [ ]:
#Eps 증가율 가져오기 함수

def make_fr_dataframe(firm_code):
    fr_url = 'https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode=' + firm_code
    fr_page = requests.get(fr_url)
    fr_tables = pd.read_html(fr_page.text)
    
    temp_df = fr_tables[0]
    
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['EPS증가율계산에 참여한 계정 펼치기']]
    temp_df.index = ['EPS증가율']   #행 이름 간편하게
    return temp_df

In [ ]:
for num, code in enumerate(mediumlist['종목코드']): # eps가져오기
    try:
        print(num, code)
        time.sleep(1)
        try:
            fr_df = make_fr_dataframe(code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            fr_df = make_fr_dataframe(code)
        fr_df_changed = change_df(code, fr_df)
        if num == 0 :
            total_fr = fr_df_changed
        else:
            total_fr = pd.concat([total_fr, fr_df_changed])
    except ValueError:
        continue
    except KeyError:
        continue

In [ ]:
total_fr

In [ ]:
total_fr.to_csv('EPS증가율.csv') #eps증가율 데이터저장

# 저변동성 데이터 구하기






In [23]:
#저변동성 데이터를 구하기 위해 중형주데이터 사용
#중형주 데이터에 붙어있는 A제거
mediumlist_t = mediumlist['종목코드'].apply(lambda x: x.replace('A',''))



In [27]:
#pykrx 사용 == 종가데이터 가져오기
#per과 eps데이터, 기관매수대금 데이터의 범위에 맞춰서 데이터를 가져와야함.

price_df = pd.DataFrame()

#tickers = stock.get_market_ticker_list(market='ALL')
tickers = mediumlist_t

for i, ticker in enumerate(tickers):
    print(i, ticker)
    try:
        temp = stock.get_market_ohlcv_by_date("20170101", "20210630", ticker) #
        price_df['A'+ticker] = temp['종가']
        time.sleep(0.1)
    except requests.exceptions.Timeout:
        time.sleep(60)
        temp = stock.get_market_ohlcv_by_date("20170101", "20210630", ticker)
        price_df['A'+ticker] = temp['종가']

0 196170
1 010620
2 036460
3 036490
4 039490
5 088350
6 020150
7 081660
8 023530
9 112610
10 272210
11 282330
12 336260
13 008560
14 047810
15 001040
16 012750
17 026960
18 204320
19 285130
20 002380
21 145020
22 047050
23 011210
24 253450
25 007070
26 030000
27 014680
28 004170
29 010060
30 001440
31 000990
32 375500
33 000080
34 058470
35 066970
36 064350
37 138040
38 138930
39 003090
40 357780
41 012510
42 009240
43 240810
44 000880
45 012450
46 000060
47 064760
48 001450
49 006260
50 095700
51 032500
52 004800
53 017800
54 001230
55 073240
56 069960
57 294870
58 016380
59 007310
60 120110
61 069620
62 237690
63 052690
64 111770
65 004370
66 000240
67 051600
68 298050
69 108320
70 011000
71 278280
72 010780
73 950210
74 005850
75 268600
76 010120
77 034230
78 000210
79 098460
80 004000
81 078340
82 042700
83 336370
84 013890
85 005250
86 178920
87 181710
88 139130
89 048260
90 006650
91 185750
92 000150
93 007390
94 200130
95 009900
96 056190
97 001740
98 030190
99 031430
100 175330

In [28]:
price_df.to_csv('가격데이터.csv') #(종가) 가격데이터 저장


In [29]:
price_df

,A196170,A010620,A036460,A036490,A039490,A088350,A020150,A081660,A023530,A112610,...,A069260,A105630,A000070,A002350,A079550,A010050,A268280,A280360,A082640,A192400
날짜,,,,,,,,,,,,,,,,,,,,,
2017-01-02,5391,33589,48300,180000,70800,6440,12393,14180,198495,8209,...,21700,26900,118500,13050,81700,644,NaN,NaN,12600,99295
2017-01-03,5525,34291,47600,184500,70900,6390,12586,14080,197120,8375,...,21400,26900,116500,13500,82900,660,NaN,NaN,12300,98909
2017-01-04,5500,32887,47850,180800,70300,6570,12296,14100,192077,8303,...,21400,26900,115000,13350,83800,661,NaN,NaN,11800,98523
2017-01-05,5491,32737,47200,177300,70300,6460,12586,14120,195744,8375,...,21300,25550,116000,13450,82200,661,NaN,NaN,11150,101614
2017-01-06,5375,33088,47750,180900,70700,6460,12586,14300,197120,8327,...,21600,25450,114500,13650,83200,668,NaN,NaN,11150,103545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-24,80900,85900,37050,320300,130000,3715,71700,54300,115000,79000,...,23050,23000,107500,9270,41850,1025,178500.0,140000.0,5610,127000
2021-06-25,83200,87700,37450,325000,130000,3840,72100,54700,116500,78100,...,22900,23350,107500,9420,41800,1045,178500.0,141500.0,5610,126000
2021-06-28,82200,87800,37300,340800,135000,3870,72500,58700,116000,76900,...,23400,24100,111500,9350,42150,1040,180500.0,141000.0,5720,125500


In [36]:
243+(61*12)

975

In [11]:
price_df = pd.read_csv('가격데이터.csv')

keras_df = pd.DataFrame() #데이터 프레임 생성

for i in price_df.columns[1:] : #여기서 데이터의 칼럼명은 종목코드 들이다.
    
    kenta = price_df.loc[:,i].pct_change() # pandas에 내장된 수익률 구하기 함수 , 현재행과 다음행을 계산하여 수익률을 만듦
    kenta = pd.DataFrame(kenta) #만든 데이터를 데이터 프레임화 시킨 후 keras_df와 병합
    keras_df = pd.concat([keras_df,kenta],axis=1)

#저변동성 == 표준편차
keras_df_1 = keras_df.iloc[:243,:].std()     #거래대금 데이터가 총 13개 (3개월 단위)
keras_df_2 = keras_df.iloc[61:243+61,:].std()  #1년은 243개 행 3개월은 61
keras_df_3 = keras_df.iloc[61*2:243+(61*2),:].std()
keras_df_4 = keras_df.iloc[61*3:243+(61*3),:].std() #아무생각없이 반복문 안쓰고 만들어벼렸음
keras_df_5 = keras_df.iloc[61*4:243+(61*4),:].std()
keras_df_6 = keras_df.iloc[61*5:243+(61*5),:].std()
keras_df_7 = keras_df.iloc[61*6:243+(61*6),:].std()
keras_df_8 = keras_df.iloc[61*7:243+(61*7),:].std()
keras_df_9 = keras_df.iloc[61*8:243+(61*8),:].std()
keras_df_10 = keras_df.iloc[61*9:243+(61*9),:].std()
keras_df_11 = keras_df.iloc[61*10:243+(61*10),:].std()
keras_df_12 = keras_df.iloc[61*11:243+(61*11),:].std()
keras_df_13 = keras_df.iloc[61*12:243+(61*12),:].std()



#데이터 프레임화
keras_df_1 = pd.DataFrame(keras_df_1)
keras_df_2 = pd.DataFrame(keras_df_2)
keras_df_3 = pd.DataFrame(keras_df_3)
keras_df_4 = pd.DataFrame(keras_df_4)
keras_df_5 = pd.DataFrame(keras_df_5)
keras_df_6 = pd.DataFrame(keras_df_6)
keras_df_7 = pd.DataFrame(keras_df_7)
keras_df_8 = pd.DataFrame(keras_df_8)
keras_df_9 = pd.DataFrame(keras_df_9)
keras_df_10 = pd.DataFrame(keras_df_10)
keras_df_11 = pd.DataFrame(keras_df_11)
keras_df_12 = pd.DataFrame(keras_df_12)
keras_df_13 = pd.DataFrame(keras_df_13)

keras_df_volacity = pd.concat([keras_df_1,keras_df_2,keras_df_3,keras_df_4,keras_df_5,keras_df_6
                              ,keras_df_7,keras_df_8,keras_df_9,keras_df_10,keras_df_11,keras_df_12,keras_df_13],axis=1)


#realvolacity = keras_df_volacity.loc[mediumlist['종목코드'],:] #중형주에 해당하는 행만 뽑아오기


FileNotFoundError: [Errno 2] No such file or directory: '가격데이터.csv'

In [ ]:
realvolacity

In [ ]:
realvolacity.to_csv("volacity.csv") #저변동성 지수 저장

# 대신증권 API 사용하여 거래량과 기관순매수 데이터 가져오기
# API 사용 코드 참고

In [5]:
import win32com.client
import sqlite3
import time
import os
import pandas as pd
import numpy as np
import sys

In [6]:
# 연결 여부 체크
def Connection():
    objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    bConnect = objCpCybos.IsConnect
    if (bConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        exit()
    else:
        print("PLUS가 정상적으로 연결되었음")
    return 
Connection()

PLUS가 정상적으로 연결되었음


In [7]:
def get_stock_list():
    
    Connection()
    print("종목을 가져옵니다")
    objCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
    
    code_list = objCpCodeMgr.GetStockListByMarket(1) #코스피
    code_list2 = objCpCodeMgr.GetStockListByMarket(2) #코스닥

    name_list = []
    name_list2 = []

    for i, code in enumerate(code_list):
        name1 = objCpCodeMgr.CodeToName(code)
        name_list.append(name1)

    for i, code in enumerate(code_list2):
        name2 = objCpCodeMgr.CodeToName(code)
        name_list2.append(name2)
    
    df_kospi = pd.DataFrame([code_list,name_list])
    df_kosdaq = pd.DataFrame([code_list2,name_list2])
    df_kospi = df_kospi.transpose()
    df_kosdaq = df_kosdaq.transpose()
    df_kospi.columns = ['code','name']
    df_kosdaq.columns = ['code','name']
    
    
    return (df_kospi, df_kosdaq)

kospi_list, kosdaq_list = get_stock_list()

PLUS가 정상적으로 연결되었음
종목을 가져옵니다


In [20]:
def RequestDT(code):
    
    def RequestData(objStockChart):
        
        objStockChart.BlockRequest()

        rqStatus = objStockChart.GetDibStatus()
        rqRet = objStockChart.GetDibMsg1()
        #print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            exit()

        length = objStockChart.GetHeaderValue(3)
        
        for i in range(length):
            dates.append(objStockChart.GetDataValue(0, i))
            
            A1.append(objStockChart.GetDataValue(2, i))
            A2.append(objStockChart.GetDataValue(3, i))

        return True

    # 폴더 없으면 생성 
    path = "./day_data_kospi"
    if not os.path.isdir(path):                                                           
        os.mkdir(path)
    
    objStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
    
    g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
    g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')

    name = g_objCodeMgr.CodeToName(code)
    
    #   이름에 / 있으면 빼주기  
    #--------------------------------------------------
    if name.find('/') != -1:
        name = name.split('/')[0] + name.split('/')[1]
    else:
        name = name
    #---------------------------------------------------
    
    dates = []
    A1 = []
    A2 = []


    
    
    objStockChart.SetInputValue(0, code)  # 종목코드
    
    
   # objStockChart.SetInputValue(1, ord('2'))  # 개수로 받기
    
    
    # 기간으로 받을 경우
    #-------------------------------------------------------
    objStockChart.SetInputValue(1, ord('1'))  # 개수로 받기
    startdate = '20171001'
    enddate = '20201230'
    objStockChart.SetInputValue(2, enddate)
    objStockChart.SetInputValue(3, startdate)
    #-------------------------------------------------------
    
    
    #objStockChart.SetInputValue(4, 916)  # 조회 개수
    objStockChart.SetInputValue(5, [0, 1,8,20])  # 요청항목 - 날짜, 시간,시가,고가,저가,종가,거래량 # 거래량 ,기관순매수량
    objStockChart.SetInputValue(6, ord('D'))  # '차트 주기 - 분/틱
    objStockChart.SetInputValue(9, ord('1'))  # 수정주가 사용
    
    
    ret = RequestData(objStockChart)
    
    # 통신확인
    #-------------------------------------------------------
    if ret == False:
        print('False')
        exit()
    #-------------------------------------------------------
    
    
    # 데이터 연속 조회
    #-------------------------------------------------------
    NextCount = 0
    time.sleep(0.3)
    while objStockChart.Continue:
        
        NextCount+=1;
        if (NextCount > 500):
            break
            
        ret = RequestData(objStockChart)
        if ret == False:
            print('False')
            exit()
            
        time.sleep(0.3)
    #-------------------------------------------------------
        
    # 받은 데이터를 데이터프레임으로 만들기 및 저장
    #-------------------------------------------------------
    df = pd.DataFrame({"date":dates,"거래량":A1,"기관순매수":A2})
    df = df.set_index('date')
    charfile = path + "/" + name + '.csv'
    df.to_csv(charfile,encoding='utf-8-sig')
    #-------------------------------------------------------
    
    return df

df = RequestDT('A005930') #  A035720
df

,거래량,기관순매수
date,,
20201230,29417421,1204866
20201229,30339449,-6166523
20201228,40085044,2397653
20201224,32502870,3706372
20201223,19411326,1824496
...,...,...
20171016,9769950,-633900
20171013,12601650,117100
20171012,13890700,-1020400


# 기관매수대금 데이터 가져오기

In [21]:
# using RequestDT(code)

mediumlist = pd.read_csv("중형주리스트.csv")


def makingbuy(x) :
    
    ratio = []

    for j in range(13):
        a = df[61*j :61*(j+1)][["거래량","기관순매수"]].sum()[1]/df[61*j:61*(j+1)][["거래량","기관순매수"]].sum()[0]
        ratio.append(a)
        
    b = pd.DataFrame(ratio)
    
    return b



for num , i in enumerate(mediumlist['종목코드']) :
    df = RequestDT(i)    
    b = makingbuy(i)
    
    b.columns = [i]
    
    if num == 0 :
        total_df = b
        
    else :
        total_df = pd.merge(total_df,b,how='outer',left_index=True, right_index=True)
    
total_df #오류메시지는 결측치생성에 따라 생성되는 것 (거래량이 0인 경우들) # 오류가 발생해도 결과값은 제대로 나온다 걱정 ㄴㄴ

C:\Users\82108\.conda\envs\py37_win32\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
C:\Users\82108\.conda\envs\py37_win32\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  # This is added back by InteractiveShellApp.init_path()


,A196170,A010620,A036460,A036490,A039490,A088350,A020150,A081660,A023530,A112610,...,A183190,A067000,A029960,A347860,A950160,A018000,A001510,A321550,A002310,A003060
0,0.017803,-0.008420,0.016213,0.009042,-0.040183,0.016250,0.005524,-0.032363,0.029032,-0.002179,...,-0.053144,-0.020614,-0.001672,-0.011494,NaN,-0.002703,-0.003497,-0.009007,-0.034739,-0.001140
1,-0.011156,-0.072101,-0.079891,-0.065195,-0.044367,0.002664,-0.010073,-0.085665,-0.063555,-0.024458,...,-0.115268,-0.009107,-0.003599,NaN,NaN,-0.004423,-0.000187,0.000458,-0.089896,-0.000839
2,-0.007357,-0.039067,-0.078665,0.006278,0.079965,-0.003524,0.010046,-0.053911,-0.036045,0.008319,...,-0.112924,0.007195,-0.019211,NaN,NaN,-0.013951,0.005680,-0.002861,-0.006566,0.000364
3,0.002693,-0.069419,-0.128821,-0.041880,-0.124672,-0.054573,-0.051900,-0.013554,-0.048818,0.091224,...,-0.158192,-0.062123,0.021951,NaN,NaN,0.017459,0.099964,-0.013276,-0.004706,0.000025
4,-0.005137,0.014972,-0.102343,-0.066574,0.156031,0.040123,0.012553,0.032715,0.067747,-0.018061,...,-0.183231,0.000215,0.035568,NaN,NaN,-0.006210,-0.001735,-0.025320,0.007553,-0.000505
5,0.001349,-0.020747,-0.085071,0.051846,-0.138156,0.004602,0.023873,0.102536,0.000043,-0.102204,...,-0.030555,-0.000152,0.011576,NaN,NaN,-0.010186,0.007862,NaN,-0.095414,-0.016649
6,-0.041036,-0.122382,0.013606,-0.101974,-0.047919,-0.031173,0.046879,-0.008114,0.006456,0.074689,...,-0.108919,-0.000082,0.008848,NaN,-0.001402,-0.099505,0.009446,NaN,0.035976,-0.004660
7,-0.025430,-0.075651,0.027840,0.037844,-0.037409,-0.037765,-0.034964,-0.067415,-0.053623,0.180681,...,0.069675,0.002684,-0.010316,NaN,-0.033182,-0.032472,-0.003692,NaN,-0.010547,-0.001442
8,0.023134,-0.034101,-0.082003,-0.106054,-0.098416,-0.140833,-0.009668,-0.091403,-0.006052,0.057239,...,-0.051678,-0.002936,0.007681,NaN,-0.000159,-0.020853,-0.030707,NaN,0.010220,0.001739
9,0.000899,0.057364,-0.049115,-0.084622,-0.040277,-0.070214,0.023466,-0.053643,-0.040435,0.120560,...,0.043769,0.008078,-0.014414,NaN,0.012379,-0.006659,-0.001535,NaN,-0.133069,-0.007014


In [22]:
total_df.to_csv("기관매수.csv") #기관매수량 저장 주의! 역순으로 되어있음(최근 부터 시작)

# 딥러닝을 위한 데이터 전처리

In [ ]:
per_rawdata = pd.read_csv('PER.csv')
eps_rawdata = pd.read_csv('EPS증가율.csv')
volacity_rawdata = pd.read_csv('volacity.csv')
quantity_rawdata = pd.read_csv('기관매수.csv')


per_rawdata = per_rawdata.set_index('Unnamed: 0')
eps_rawdata = eps_rawdata.set_index('Unnamed: 0')
volacity_rawdata = volacity_rawdata.set_index('Unnamed: 0')

#quantity는 이미 인덱스가 바뀌어있음

qunatity_rawdata = quantity_rawdata.fillna(-1)


X_data = pd.DataFrame()

quantity_rawdata = quantity_rawdata.T

In [ ]:
quantity_rawdata

In [ ]:
# X에 해당하는 학습데이터 만들기

for i in range(4) : #2017 , 2018, 2019, 2020
    perdata = per_rawdata.iloc[1:,i+2] #per 데이터 할당

    epsdata = eps_rawdata.iloc[1:,i+2] #eps 데이터 할당
    epsdata1=epsdata.replace('흑전','0.5')    # 흑자전화과 적자전환 적자지속 에 해당하는 '문자열'을
    epsdata1=epsdata1.replace('적지','-80.0')  # -80과 0.5 의 임의의 숫자로 할당 == 사실상 고려하지 않기를 위해 극단적인 값으로 할당
    epsdata1=epsdata1.replace('적전','-80.0')
    
    if i <=2 :
        for j in range(4):
            volacitydata = volacity_rawdata.iloc[:,(4*i)+j] #저변동성 데이터 할당
        
            quantitydata = quantity_rawdata.iloc[1:,(4*i)+j]
    
            final_data = pd.concat([perdata,epsdata1,volacitydata,quantitydata],axis=1) #데이터 프레임 합치기
            final_data.columns = ['PER','EPS증가율','저변동성','QUANTITY'] #칼럼 이름 지정
            final_data = final_data.apply(pd.to_numeric,errors = 'coerce') #계산이 가능하도록 numeric으로 변환
    
    # Z-SCORE 형태로 표준화를 시켜줌 -- 서로간의 단위가 다르기 때문
    
            final_data['PER스코어'] = (final_data['PER'] - final_data['PER'].mean())/final_data['PER'].std()
            final_data['EPS스코어'] = -(final_data['EPS증가율'] - final_data['EPS증가율'].mean())/final_data['EPS증가율'].std()
            final_data['VOL스코어'] = (final_data['저변동성'] - final_data['저변동성'].mean())/final_data['저변동성'].std()
            final_data['QUT스코어'] = -(final_data['QUANTITY'] - final_data['QUANTITY'].mean())/final_data['QUANTITY'].std()
    
    #EPS,QUT 스코어에  " - "  를 붙인이유 : PER과 저변동성 모두 낮을 수록 좋은 수치이기 때문에 이와 같이 통일하기 위해서 -를 붙임
    
            X_data = pd.concat([X_data,final_data])
        
        
    else :
        volacitydata = volacity_rawdata.iloc[:,12] #저변동성 데이터 할당
        
        quantitydata = quantity_rawdata.iloc[1:,12]
    
        final_data = pd.concat([perdata,epsdata1,volacitydata,quantitydata],axis=1) #데이터 프레임 합치기
        final_data.columns = ['PER','EPS증가율','저변동성','QUANTITY'] #칼럼 이름 지정
        final_data = final_data.apply(pd.to_numeric) #계산이 가능하도록 numeric으로 변환
    
    
        final_data['PER스코어'] = (final_data['PER'] - final_data['PER'].mean())/final_data['PER'].std()
        final_data['EPS스코어'] = -(final_data['EPS증가율'] - final_data['EPS증가율'].mean())/final_data['EPS증가율'].std()
        final_data['VOL스코어'] = (final_data['저변동성'] - final_data['저변동성'].mean())/final_data['저변동성'].std()
        final_data['QUT스코어'] = -(final_data['QUANTITY'] - final_data['QUANTITY'].mean())/final_data['QUANTITY'].std()
    
    
        X_data = pd.concat([X_data,final_data])

In [ ]:
X_data

## 딥러닝을 효과적으로 이용하고 해석하기 위해서 특정값을 예측하기 보다는 일정 범위안에 들었는지 분류하는 것이 더 용이하다고 생각하여
## 분류 형태 (클래스)로 Y (종속변수) 값을 설정하였다

In [ ]:
#y데이터 만들기 
price_df = pd.read_csv('가격데이터.csv')
price_dfx = price_df.iloc[:,1:]   #중형주에 해당하는 수익률 데이터만 가져오기
price_dfx = price_dfx.apply(pd.to_numeric) #계산이 가능하도록 바꿔줌

#특정 날짜 이후의 값과 비교하여 수익률을 계산
#23 1개월  #61 3개월  <- 원하는 개월수에 맞추기
#현재 3개월 이후의 수익률을 y값으로 활용하고 있다.

# 10위 안에 들었으면 1 , 아니면 0
def finite(x):
    if x <= 10 :
        return 1
    else :
        return 0


Y_data = pd.DataFrame()

for i in range(13):
    kunka = (price_dfx.iloc[61*(i+1)+186,1:]-price_dfx.iloc[61*i + 186,1:])/price_dfx.iloc[61*i + 186,1:] 
    kunka =  pd.DataFrame(kunka)
    kunkax = kunka.loc[mediumlist['종목코드'],:]
    kunkax['rank'] = kunkax.rank()   
    kunkax['zscore']= kunkax[0]-kunkax[0].mean()/kunkax[0].std()
    kunkax['class'] = kunkax['rank'].apply(finite) # finite 함수 적용

    Y_data = pd.concat([Y_data,kunkax])

Y_data

In [ ]:

# class데이터만 X데이터와 합치고 결측치를 제거한 후 저장

X_data['y'] = Y_data['class']
X_data = X_data.dropna(how='any')
X_data.isnull().sum().sum()

In [ ]:
X_data

In [ ]:
X_data.to_csv("Dataset.csv")

# 딥러닝 적용하기

In [ ]:
#############딥러닝 가동##################   X_data Y_rank_data Y_zscore_data
X_data = pd.read_csv("Dataset.csv")
X_data = X_data.set_index("Unnamed: 0")
X_data

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import numpy
import pandas as pd
import tensorflow as tf


X = X_data.values[:,4:8]
Y = X_data.values[:,8]

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1, random_state=0)

model = Sequential()
model.add(Dense(30,activation='relu',input_dim=4))
model.add(Dense(20,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


model.fit(X_train,Y_train,epochs=100,batch_size=100)


In [ ]:
model.evaluate(X_test, Y_test)

# 테스트 파일을 가져와서 적용

In [ ]:

testfile = pd.read_csv("testfile.csv")

testfile1 = testfile.iloc[:,1:]
testfile2 = testfile1.apply(pd.to_numeric)

malis = model.predict(testfile2.values[:,3:6])
malis1 = pd.DataFrame(malis)
malis1.sort_values(by=0,ascending=False).head(10)

# 백테스팅 - kqr 참고자료 내용 그대로 적용시킨 결과이다.(리밸런싱x)

In [ ]:
import matplotlib.pyplot as plt

price_df = pd.read_csv(r"C:\Users\82108\Downloads\KQR\페이즈2\가격데이터.csv")

blist = [    'A017800', #예시
      'A004800',
    'A081000',
      'A003000',
    'A012510',
    'A241590',
    'A020150',
    'A005070',
    'A003090',
      'A181710',
    'A052690',
    'A226320'
]

price_df.index = price_df['날짜']

price_df1 = price_df.loc['2019-01-02':,blist] # 예시의 2019년 1월 2일부터 blist에 해당하는 가격데이터만 가져오기




In [ ]:
#포트폴리오 구성
pf_stock_num = {}   #각 종목을 몇개씩 살지
stock_amount = 0    #주식매수금액

for code in price_df1.columns:
    temp = int(10000000 / price_df1[code][0])
    pf_stock_num[code] = temp 
    stock_amount=stock_amount + temp * price_df1[code][0]
    

cash_amount = 100000000 - stock_amount   #보유현금금액
print('보유종목수: ', pf_stock_num)
print('주식보유금액: ', stock_amount)
print('현금보유금액: ', cash_amount)

In [ ]:

stock_pf = 0
for code in price_df1.columns:
    stock_pf = stock_pf + price_df1[code] * pf_stock_num[code]
stock_pf

In [ ]:
#일일 수익률 및 누적 수익률 구하기

low_pbr_bt = pd.DataFrame({'주식포트폴리오' : stock_pf[:'2020-03-31']})
low_pbr_bt['현금포트폴리오'] = [cash_amount] * len(low_pbr_bt)
low_pbr_bt['종합'] = low_pbr_bt['주식포트폴리오'] + low_pbr_bt['현금포트폴리오']
low_pbr_bt['일일수익률'] = low_pbr_bt['종합'].pct_change()
low_pbr_bt['누적수익률'] = low_pbr_bt['종합']/100000000 -1
low_pbr_bt

In [ ]:
#그림

plt.figure(figsize=(10,6))
low_pbr_bt['누적수익률'].plot()
plt.xlabel('Date')
plt.title('Cumulative Returns')
plt.show()